In [1]:
import os
import pandas as pd
import numpy as np

from controllers.inference_controllers.sandbox import Sandbox
from controllers.inference_controllers.generator import Generator
from controllers.inference_controllers.optimiser import Optimiser

from toolboxes.plotting_toolbox.domain import Domain
from toolboxes.inference_toolbox.parameter import Parameter
from toolboxes.inference_toolbox.model import Model
from toolboxes.inference_toolbox.likelihood import Likelihood

#current_directory = os.getcwd()
#if current_directory != '/project/':
    #os.chdir('/project/')

In [2]:
data_params = {
    'data_type': 'simulated_data',
    'model': {
        'model_select': 'sarah_model',
        'model_params':{
            'u': 7.5
        },
        'inference_params':{
            'I_y': 0.1,
            'I_z': 0.1,
            'Q': 3e13,
            'sigma': 1,
            'x_0': 0,
            'y_0': 0,
            'z_0':0
        }
    },
    'domain':{
        'domain_select': 'cone_from_source_z_limited',
        'domain_params': {
            'r': 100,
            'theta': np.pi/8,
            'source': [0,0,0]
        },
        'resolution': 20
    },
    'noise_dist': 'gaussian',
    'noise_level': 1,
    'output_header': 'Concentration',
}

# data_params = {
#     'data_type': 'normalised_data',
#     'data_select': 'GBR_data',
#     'normaliser_select': 'GBR_normaliser',
#     'normaliser_params':{
#         'experiments_list': [
#             'Exp1',
#             'Exp2',
#             'Exp3',
#             'Exp4',
#             'Exp5',
#             'Exp6',
#             'Exp7',
#             'Exp8',
#             'Exp9',
#             'Exp10',
#             'Exp11',
#             'Exp12',
#         ],
#         'meta_data_select': 'GBR_data_summary',
#         'input_header': 'Concentration'
#     },
#     'log':True,
#     'output_header': 'Concentration',
#     'gridding': [100,100,25]
# }



default_params = {
    'infered_params':pd.Series({
        'model_params':pd.Series({
            'I_y': Parameter('I_y', prior_select = 'gamma', default_value=0.1).add_prior_param('mu', 0.1).add_prior_param('sigma',0.1),
            'I_z': Parameter('I_z', prior_select = 'gamma', default_value=0.1).add_prior_param('mu', 0.1).add_prior_param('sigma',0.1),
            'Q': Parameter('Q', prior_select = 'gamma', default_value=3e13).add_prior_param('mu', 3e13).add_prior_param('sigma',1e13),
            'x_0': Parameter('x_0', prior_select = 'uniform', default_value=0).add_prior_param('low', -3).add_prior_param('high',3),
            'y_0': Parameter('y_0', prior_select = 'uniform', default_value=0).add_prior_param('low', -3).add_prior_param('high',3),
            'z_0': Parameter('z_0', prior_select = 'uniform', default_value=0).add_prior_param('low', -3).add_prior_param('high',3),
        }),
        'likelihood_params':pd.Series({
            'sigma': Parameter('sigma', prior_select = 'gamma', default_value=1).add_prior_param('mu', 1).add_prior_param('sigma',1)
        })
    }),
    'model':Model('sarah_model').add_model_param('u',7.5),
    'likelihood': Likelihood('gaussian'),
    'sampler': {
        'n_samples': 1000,
        'n_chains': 3,
        'thinning_rate': 1
    }
}

results_name = 'diffusioncoeff_sensitivityanalysis_inferred'

In [3]:
sandbox = Sandbox(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

visualiser = sandbox.run()
visualiser.get_summary()
visualiser.get_traceplot()
visualiser.get_autocorrelations()

domain = Domain('cone_from_source_z_limited', resolution=50)
domain.add_domain_param('r', 1000)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,0])

visualiser.visualise_results(domain = domain, name = 'small_scale_3D_plots', title='Log Concentration of Droplets', log_results=False)
visualiser.animate(name = 'small_scale_3D_plots')

sample: 100%|██████████| 1250/1250 [00:01<00:00, 728.45it/s, 15 steps of size 3.93e-01. acc. prob=0.92]


Creating instance


In [4]:
generator = Generator(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

In [5]:
analysis_iterations = {
    'parameters_1':
    [
        'sigma_mu'
    ],
    'parameters_2':
    [
        'sigma_sigma'

    ],
    'values_1':
    [
        np.array([0.5, 1])     
    ],

    'values_2':
    [
        np.array([0.5, 1]) 
    ]
}

for i in range(len(analysis_iterations['parameters_1'])):
    parameter_1 = analysis_iterations['parameters_1'][i]
    parameter_2 = analysis_iterations['parameters_2'][i]

    print('Working on varying ' + parameter_1 + '...')
    values_1 = analysis_iterations['values_1'][i]
    values_2 = analysis_iterations['values_2'][i]
    inputs = generator.vary_two_parameters(parameter_1, parameter_2, values_1, values_2, scale_1='linear', scale_2='linear', plot=True)

Working on varying sigma_mu...
Generating instance 1...


sample: 100%|██████████| 1250/1250 [00:01<00:00, 761.60it/s, 31 steps of size 1.79e-01. acc. prob=0.88]


Generating instance 2...


sample: 100%|██████████| 1250/1250 [00:02<00:00, 450.95it/s, 11 steps of size 1.87e-01. acc. prob=0.84]


Generating instance 3...


sample: 100%|██████████| 1250/1250 [00:03<00:00, 331.93it/s, 23 steps of size 2.30e-01. acc. prob=0.91]


Generating instance 4...


sample: 100%|██████████| 1250/1250 [00:04<00:00, 309.83it/s, 15 steps of size 1.87e-01. acc. prob=0.82]
